## Load image slices and plot axial, sagittal and coronal images

Based on pydicom example: [Load CT slices and plot axial, sagittal and coronal images](https://pydicom.github.io/pydicom/stable/auto_examples/image_processing/reslice.html#sphx-glr-auto-examples-image-processing-reslice-py)

This example illustrates loading a scan with XNATpy and pydicom, building a 3D image, and reslicing it in different planes.

In [1]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
import xnat
import time
import os

from ipywidgets import interact
from scipy import ndimage

In [2]:
session = xnat.connect()
project = session.projects['UPENN-GBM']
subject = project.subjects['UPENN-GBM-00001']
experiment = subject.experiments['UPENN-GBM-00001_MR_1']
scan = experiment.scans['2']

In [3]:
# load the DICOM files
files = []
for file in scan.files.values():
    if file.uri.endswith("dcm"):
        files.append(pydicom.dcmread(file.open()))
        
print("file count: {}".format(len(files)))

# skip files with no SliceLocation (eg scout views)
slices = []
skipcount = 0
for f in files:
    if hasattr(f, 'SliceLocation'):
        slices.append(f)
    else:
        skipcount = skipcount + 1

print("skipped, no SliceLocation: {}".format(skipcount))

# ensure they are in the correct order
slices = sorted(slices, key=lambda s: s.SliceLocation)

# pixel aspects, assuming all slices are the same
ps = slices[0].PixelSpacing
ss = slices[0].SliceThickness
ax_aspect = ps[1]/ps[0]
sag_aspect = ps[1]/ss
cor_aspect = ss/ps[0]

# create 3D array
img_shape = list(slices[0].pixel_array.shape)
img_shape.append(len(slices))
img3d = np.zeros(img_shape)

# fill 3D array with the images from the files
for i, s in enumerate(slices):
    img2d = s.pixel_array
    img3d[:, :, i] = img2d


def browse_images_ax(img3d, img_shape, ax_aspect):
    n = img_shape[2]
    def view_image(i):
        a = plt.subplot()
        plt.imshow(img3d[:, :, i], cmap=plt.cm.bone, interpolation='nearest')
        a.set_aspect(ax_aspect)
        plt.title('Image: %s' % i)
        plt.show()
    interact(view_image, i=(0,n-1))
    
def browse_images_sag(img3d, img_shape, sag_aspect):
    n = img_shape[1]
    def view_image(i):
        a = plt.subplot()
        rotated_img = ndimage.rotate(img3d[:, i, :], 90)
        plt.imshow(rotated_img, cmap=plt.cm.bone, interpolation='nearest')
        a.set_aspect(1/sag_aspect)
        plt.title('Image: %s' % i)
        plt.show()
    interact(view_image, i=(0,n-1))  
    
def browse_images_cor(img3d, img_shape, cor_aspect):
    n = img_shape[0]
    def view_image(i):
        a = plt.subplot()
        rotated_img = ndimage.rotate(img3d[i, :, :].T, 180)
        plt.imshow(rotated_img, cmap=plt.cm.bone, interpolation='nearest')
        a.set_aspect(cor_aspect)
        plt.title('Image: %s' % i)
        plt.show()
    interact(view_image, i=(0,n-1))  

file count: 60
skipped, no SliceLocation: 0


In [4]:
browse_images_ax(img3d, img_shape, ax_aspect)
browse_images_sag(img3d, img_shape, sag_aspect)
browse_images_cor(img3d, img_shape, cor_aspect)

interactive(children=(IntSlider(value=29, description='i', max=59), Output()), _dom_classes=('widget-interact'…

interactive(children=(IntSlider(value=95, description='i', max=191), Output()), _dom_classes=('widget-interact…

interactive(children=(IntSlider(value=127, description='i', max=255), Output()), _dom_classes=('widget-interac…